In [43]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from datetime import datetime
import numpy as np
import json


import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [6]:
%%time
df_2021 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2021.csv', encoding='cp949')

CPU times: total: 54.5 s
Wall time: 54.6 s


In [48]:
%%time
df_2022 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2022.csv', encoding='cp949')

CPU times: total: 33.4 s
Wall time: 33.5 s


In [7]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [9]:
print('df_2021')
df_astype(df_2021)
print('-------------'*3)

df_2021
데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1646.35 MB
감소율 : 39.2%
---------------------------------------


In [49]:
print('df_2022')
df_astype(df_2022)
print('-------------'*3)

df_2022
데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%
---------------------------------------


In [61]:
df_2022[df_2022['MAC_add'] == '7C573CCF497E']

Path_ID     CLIENTMAC       MAC_add      STARTTIMESTAMP  \
311            783     000000000  7C573CCF497E 2022-10-01 16:30:00   
6748         21591  00012E10B9ED  7C573CCF497E 2022-10-21 00:40:00   
21953        51520  000270E3B86A  7C573CCF497E 2022-05-20 09:40:00   
21963        51530  000270E3B86A  7C573CCF497E 2022-06-20 10:50:00   
21975        51633  000270E3B9C3  7C573CCF497E 2022-03-21 17:20:00   
...            ...           ...           ...                 ...   
28919420  73244576  FCFD7B5118EA  7C573CCF497E 2022-06-27 08:50:00   
28919450  73244705  FCFF9B4B4D76  7C573CCF497E 2022-03-20 20:30:00   
28919592  73249027  FD7C75731000  7C573CCF497E 2022-03-16 15:20:00   
28919651  73251402  FDC30F4ECB6F  7C573CCF497E 2022-07-06 15:00:00   
28919711  73253163  FDF62F723EC1  7C573CCF497E 2022-08-07 16:30:00   

                ENDTIMESTAMP  Previous_Path  Next_Path  Next_MAC  
311      2022-10-01 16:40:00            NaN        NaN       NaN  
6748     2022-10-21 00:50:00            NaN        NaN       NaN  
21953    2022-05-20 09:50:00            NaN        NaN       NaN  
21963    2022-06-20 11:00:00        51529.0        NaN       NaN  
21975    2022-03-21 17:30:00        51632.0        NaN       NaN  
...                      ...            ...        ...       ...  
28919420 2022-06-27 09:00:00            NaN        NaN       NaN  
28919450 2022-03-20 20:40:00            NaN        NaN       NaN  
28919592 2022-03-16 15:30:00            NaN        NaN       NaN  
28919651 2022-07-06 15:10:00            NaN        NaN       NaN  
28919711 2022-08-07 16:40:00            NaN        NaN       NaN  

[1158799 rows x 8 columns]

In [62]:
df = df_2022[df_2022['STARTTIMESTAMP'] >= datetime.strptime(f'2022-10-01 00:00:00',
                                              '%Y-%m-%d %H:%M:%S')]

In [65]:
df_11 = df[df['MAC_add']=='7C573CCF497E']
df_11

Path_ID     CLIENTMAC       MAC_add      STARTTIMESTAMP  \
311            783     000000000  7C573CCF497E 2022-10-01 16:30:00   
6748         21591  00012E10B9ED  7C573CCF497E 2022-10-21 00:40:00   
23280        62381  000270E3C1CE  7C573CCF497E 2022-10-01 00:40:00   
23394        62495  000270E3C1CE  7C573CCF497E 2022-10-20 07:50:00   
23411        62512  000270E3C1CE  7C573CCF497E 2022-10-22 05:40:00   
...            ...           ...           ...                 ...   
28917712  73236052  FCF13621BF28  7C573CCF497E 2022-10-31 18:10:00   
28917713  73236053  FCF13621BF28  7C573CCF497E 2022-10-31 18:50:00   
28917714  73236054  FCF13621BF28  7C573CCF497E 2022-10-31 19:30:00   
28917715  73236055  FCF13621BF28  7C573CCF497E 2022-10-31 22:10:00   
28919259  73243961  FCF8AE1B5B8E  7C573CCF497E 2022-10-22 04:30:00   

                ENDTIMESTAMP  Previous_Path  Next_Path  Next_MAC  
311      2022-10-01 16:40:00            NaN        NaN       NaN  
6748     2022-10-21 00:50:00            NaN        NaN       NaN  
23280    2022-10-01 00:50:00        62380.0        NaN       NaN  
23394    2022-10-20 08:00:00            NaN        NaN       NaN  
23411    2022-10-22 05:50:00            NaN    62513.0      61.0  
...                      ...            ...        ...       ...  
28917712 2022-10-31 18:20:00            NaN        NaN       NaN  
28917713 2022-10-31 19:00:00            NaN        NaN       NaN  
28917714 2022-10-31 19:40:00            NaN        NaN       NaN  
28917715 2022-10-31 22:20:00            NaN        NaN       NaN  
28919259 2022-10-22 04:40:00            NaN        NaN       NaN  

[122074 rows x 8 columns]

In [67]:
cl_list = df_11['CLIENTMAC'].unique()

In [69]:
cl_list_num = []
for n in tqdm(cl_list):
    df_12 = df_11[df_11['CLIENTMAC']==n]
    cl_list_num.append(len(df_12))

  0%|          | 0/17990 [00:00<?, ?it/s]

In [70]:
dict_1 = {'cl_list':cl_list, 'cl_list_num':cl_list_num}
df_13 = pd.DataFrame(dict_1)
df_13

cl_list  cl_list_num
0         000000000            1
1      00012E10B9ED            1
2      000270E3C1CE            6
3      000270E3C1D1            3
4      000270E3C273            6
...             ...          ...
17985  FCE9DF871607            1
17986  FCEB42E5B31D            1
17987  FCEC1C697BE5            1
17988  FCF13621BF28          831
17989  FCF8AE1B5B8E            1

[17990 rows x 2 columns]

In [80]:
df_16 = df_13.sort_values('cl_list_num', ascending=False).head(20)

In [78]:
df_15 = df_11[df_11['CLIENTMAC']=='808C97A82C32']

In [79]:
df_15.to_csv('./10월_808C97A82C32.csv',encoding='cp949',index=False)

In [81]:
df_16.to_excel('./2022년_10월_Wi-Fi(부산은행)_이용자.xlsx')

In [46]:
def WiFi_user_counting(df_year, year):
    
    if year == 2021:
        month_list = [i for i in range(1,13)]
    elif year == 2022:
        month_list = [i for i in range(3,11)]
        
    wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

    for month in tqdm(month_list):
        if month != 12:
            df = df_year[(df_year['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{month}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S')) &
                (df_year['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{month+1}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]
        else:
            df = df_year[df_year['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{month}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S')]
        df_mac_list = df['MAC_add'].unique()

        mac_li = []
        u_ct = []
        X = []
        Y = []

        for mac in df_mac_list:
            DF = df[df['MAC_add']==mac]
            use_count = len(DF['CLIENTMAC'].unique())

            mac_li.append(mac)
            u_ct.append(use_count)
            X.append(wk.loc[mac]['경도'])
            Y.append(wk.loc[mac]['위도'])

        df_dict = {'MAC_add':mac_li, 'User_count':u_ct, 'X':X, 'Y':Y}

        df_save = pd.DataFrame(df_dict)

        # 폴더 생성
        try:
            csv_directory = f'./전처리/와이파이_개소별_사용자/csv/{year}년/'
            if not os.path.exists(csv_directory):
                os.makedirs(csv_directory)
        except OSError:
            print ('Error: Creating directory. ' +  csv_directory)

        df_save.to_csv(csv_directory + f'{year}년_{month}월.csv', encoding='cp949', index=False)



        # shp파일 저장
        df_save['geometry'] = df_save.apply(lambda row : Point([row['X'], row['Y']]), axis=1)
        df_save = gpd.GeoDataFrame(df_save, geometry='geometry', crs = epsg4326)

        # 폴더 생성
        try:
            shp_directory = f'./전처리/와이파이_개소별_사용자/shp/{year}년/{month}월/'
            if not os.path.exists(shp_directory):
                os.makedirs(shp_directory)
        except OSError:
            print ('Error: Creating directory. ' +  shp_directory)

        df_save.to_file(shp_directory + f'{year}년_{month}월.shp',encoding='cp949', driver='ESRI Shapefile')

In [47]:
WiFi_user_counting(df_2021, 2021)

  0%|          | 0/12 [00:00<?, ?it/s]

In [59]:
WiFi_user_counting(df_2022, 2022)

  0%|          | 0/8 [00:00<?, ?it/s]